# 관광객 데이터 전처리
### 데이터 불러오기

In [1]:
import pandas as pd
import itertools as ite
import openpyxl

In [18]:
# 자동화

# kto_2019.xslx 파일 불러오기
# - 컬럼명과 컬럼에 대한 데이터만 불러오기
# 미리 파일을 열어봐서 가지고 있는 데이터 형태가 같다는걸 확인함
path = './files/kto_201901.xlsx'
kto_201901 = pd.read_excel(path, header = 1,
              skipfooter = 4,  # 아래에서 행 제거
              usecols = 'A:G') # 읽어들일 컬럼 선택 

In [19]:
# 기본 정보 확인하기
kto_201901.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적      67 non-null     object
 1   관광      67 non-null     int64 
 2   상용      67 non-null     int64 
 3   공용      67 non-null     int64 
 4   유학/연수   67 non-null     int64 
 5   기타      67 non-null     int64 
 6   계       67 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 3.8+ KB


In [34]:
kto_201901.describe()
# kto_201901.agg([sum])

,관광,상용,공용,유학/연수,기타,계
count,67.00000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,26396.80597,408.208955,132.507463,477.462687,5564.208955,32979.194030
std,102954.04969,1416.040302,474.406339,2009.484800,17209.438418,122821.369969
min,0.00000,0.000000,0.000000,0.000000,16.000000,54.000000
25%,505.00000,14.500000,2.500000,17.500000,260.000000,927.000000
50%,1304.00000,45.000000,14.000000,43.000000,912.000000,2695.000000
75%,8365.00000,176.500000,38.000000,182.000000,2824.500000,14905.500000
max,765082.00000,10837.000000,2657.000000,14087.000000,125521.000000,916950.000000


In [65]:
# min 이 0인게 좀 잇네 확인해야지
condition = (kto_201901['관광'] == 0)\
           | (kto_201901['상용'] == 0)\
           | (kto_201901['공용']== 0)\
           | (kto_201901['유학/연수']== 0)
kto_201901[condition]
# 교포소계랑 교포가0이네 이게뭔고 하니
# 교포는 국가가 아니라서 기타에 올인했다 카더라 그럼 결측치는 아니네

,국적,관광,상용,공용,유학/연수,기타,계
4,마카오,2506,2,0,17,45,2570
20,이스라엘,727,12,0,9,57,805
22,우즈베키스탄,1958,561,0,407,2828,5754
38,스위스,613,18,0,19,97,747
45,그리스,481,17,4,0,273,775
46,포르투갈,416,14,0,13,121,564
51,크로아티아,226,12,0,3,250,491
54,폴란드,713,10,0,27,574,1324
59,대양주 기타,555,3,4,0,52,614
63,기타대륙,33,4,0,1,16,54


In [67]:
# 자동화

# 파일명에는 년월이 있는데 파일 안쪽만 보면 모르겟네
# 다른 파일과 합칠 수도 있느이 날짜 컬럼을 넣어주자
# 컬럼명 : 기준년월
# 값 : 2019-01

kto_201901['기준년월'] = '2019-01'
kto_201901

,국적,관광,상용,공용,유학/연수,기타,계,기준년월
0,아시아주,765082,10837,1423,14087,125521,916950,2019-01
1,일본,198805,2233,127,785,4576,206526,2019-01
2,대만,86393,74,22,180,1285,87954,2019-01
3,홍콩,34653,59,2,90,1092,35896,2019-01
4,마카오,2506,2,0,17,45,2570,2019-01
...,...,...,...,...,...,...,...,...
62,아프리카 기타,768,718,90,206,908,2690,2019-01
63,기타대륙,33,4,0,1,16,54,2019-01
64,국적미상,33,4,0,1,16,54,2019-01
65,교포소계,0,0,0,0,15526,15526,2019-01


In [78]:
# 데이터 걸러내기
# 아시아주, 아프리카주 같이 국가가 아닌 대륙별로 나뉜게 같이 들어 있다.
# 국가가 어느대륙에 들어있는지도 쓸모있지만 지금은 걸러내야한다
# 어떤 패턴으로 데이터가 들어있는지 확인
# 국적 컬럼의 값을 조회하고 고유한 값들만 출력
kto_201901['국적'].unique()

array(['아시아주', '일본', '대만', '홍콩', '마카오', '태국', '말레이시아', '필리핀', '인도네시아',
       '싱가포르', '미얀마', '베트남', '인도', '스리랑카', '파키스탄', '방글라데시', '캄보디아', '몽골',
       '중국', '이란', '이스라엘', '터키', '우즈베키스탄', '카자흐스탄', 'GCC', '아시아 기타', '미주',
       '미국', '캐나다', '멕시코', '브라질', '미주 기타', '구주', '영국', '독일', '프랑스',
       '네덜란드', '스웨덴', '스위스', '이탈리아', '덴마크', '노르웨이', '벨기에', '오스트리아', '스페인',
       '그리스', '포르투갈', '핀란드', '아일랜드', '우크라이나', '러시아', '크로아티아', '루마니아',
       '불가리아', '폴란드', '구주 기타', '대양주', '오스트레일리아', '뉴질랜드', '대양주 기타',
       '아프리카주', '남아프리카공화국', '아프리카 기타', '기타대륙', '국적미상', '교포소계', '교포'],
      dtype=object)

In [80]:
# 자동화

# 대륙의 값들을 관리하는 목록 만들기 : 기준이 없기 때문에 기준을 만들어 줍니다.
continents_list = ['아시아주', '미주', '대양주', '구주',
                   '아프리카주', '기타대륙', '교포소계']
continents_list


['아시아주', '미주', '대양주', '구주', '아프리카주', '기타대륙', '교포소계']

In [94]:
# 자동화

# 국적 데이터에서 대륙 기준값이 포함되어 있는지 확인
# 사용함수 isin()
kto_201901_contry = kto_201901[kto_201901['국적'].isin(continents_list) == False]

In [104]:
# 자동화

# 지울꺼 지웠으니 인덱스 번호 재배열 하기
# 재배열 후 kto_201901_contry_newindex 변수에 저장하기
kto_201901_contry_newindex = kto_201901_contry.reset_index(drop = True)
kto_201901_contry_newindex

,국적,관광,상용,공용,유학/연수,기타,계,기준년월
0,일본,198805,2233,127,785,4576,206526,2019-01
1,대만,86393,74,22,180,1285,87954,2019-01
2,홍콩,34653,59,2,90,1092,35896,2019-01
3,마카오,2506,2,0,17,45,2570,2019-01
4,태국,34004,37,199,96,6998,41334,2019-01
5,말레이시아,19043,95,7,99,2821,22065,2019-01
6,필리핀,14279,211,161,184,15638,30473,2019-01
7,인도네시아,14183,136,38,187,4298,18842,2019-01
8,싱가포르,8372,94,8,48,1333,9855,2019-01
9,미얀마,1304,10,31,67,3877,5289,2019-01


In [106]:
# 대륙 컬럼을 각 국가별로 적어서 만들것
# 아시아주 25개, 미주 5개, 구주 23개, 대양주 3개, 아프리카 2개, 교포 1개
print(len(kto_201901_contry_newindex)) # 60개의 데이터를 가지고 있음


60


In [110]:
# 자동화

# 아시아(25) 아메리카(5) 유럽(23) 오세아니아(3) 아프리카(2) 기타대륙(국적미상 1) 교포(1)
# 위 데이터를 알고 있으니 미리 새 컬럼에 추가할 리스트 생성
continents = ['아시아'] * 25 + ['아메리카'] * 5 \
+ ['유럽'] * 23 + ['오세아니아'] * 3 + ['아프리카'] * 2 + ['기타대륙'] + ['교포']
continents

['아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아시아',
 '아메리카',
 '아메리카',
 '아메리카',
 '아메리카',
 '아메리카',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '유럽',
 '오세아니아',
 '오세아니아',
 '오세아니아',
 '아프리카',
 '아프리카',
 '기타대륙',
 '교포']

In [113]:
kto_201901_contry_newindex['대륙'] = continents
kto_201901_contry_newindex.head()
kto_201901_contry_newindex.tail()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙
55,대양주 기타,555,3,4,0,52,614,2019-01,오세아니아
56,남아프리카공화국,368,9,1,6,616,1000,2019-01,아프리카
57,아프리카 기타,768,718,90,206,908,2690,2019-01,아프리카
58,국적미상,33,4,0,1,16,54,2019-01,기타대륙
59,교포,0,0,0,0,15526,15526,2019-01,교포


In [114]:
# 이상데이터였던 대륙을 빼고 아까워서 살리고
# 결측치도 없고 이제 뭐하지?
kto_201901_contry_newindex

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아
1,대만,86393,74,22,180,1285,87954,2019-01,아시아
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아
3,마카오,2506,2,0,17,45,2570,2019-01,아시아
4,태국,34004,37,199,96,6998,41334,2019-01,아시아
5,말레이시아,19043,95,7,99,2821,22065,2019-01,아시아
6,필리핀,14279,211,161,184,15638,30473,2019-01,아시아
7,인도네시아,14183,136,38,187,4298,18842,2019-01,아시아
8,싱가포르,8372,94,8,48,1333,9855,2019-01,아시아
9,미얀마,1304,10,31,67,3877,5289,2019-01,아시아


In [117]:
# 자동화

# 국가별로 관광을 통해서 우리나라에 들어오는 비율을 알아내서 컬럼으로 만들어 볼까?
# 국가별 관광/계 를 이용하여 '관광객 비율(%)' 이름으로 넣기
# 비율은 소수점 1자리 까지만 넣기

kto_201901_contry_newindex['관광객 비율(%)'] = \
round((kto_201901_contry_newindex['관광'] / kto_201901_contry_newindex['계']) * 100, 1)
kto_201901_contry_newindex


,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객 비율(%)
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5
4,태국,34004,37,199,96,6998,41334,2019-01,아시아,82.3
5,말레이시아,19043,95,7,99,2821,22065,2019-01,아시아,86.3
6,필리핀,14279,211,161,184,15638,30473,2019-01,아시아,46.9
7,인도네시아,14183,136,38,187,4298,18842,2019-01,아시아,75.3
8,싱가포르,8372,94,8,48,1333,9855,2019-01,아시아,85.0
9,미얀마,1304,10,31,67,3877,5289,2019-01,아시아,24.7


In [124]:
# 자동화

# 전세계 관광객 임국객수 대비 국가별 관광객 비율을 계산하여 컬럼생성
# 컬럼명 : 전체비율(%)
# 답 : 각국가 관광객수 / 전체관광객수 *100
# 소수점 1자리까지
kto_201901_contry_newindex['전체비율(%)'] = \
round(kto_201901_contry_newindex['관광'] / kto_201901_contry_newindex['관광'].sum() * 100, 1)
kto_201901_contry_newindex

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객 비율(%),전체비율(%)
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3,22.5
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2,9.8
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5,3.9
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5,0.3
4,태국,34004,37,199,96,6998,41334,2019-01,아시아,82.3,3.8
5,말레이시아,19043,95,7,99,2821,22065,2019-01,아시아,86.3,2.2
6,필리핀,14279,211,161,184,15638,30473,2019-01,아시아,46.9,1.6
7,인도네시아,14183,136,38,187,4298,18842,2019-01,아시아,75.3,1.6
8,싱가포르,8372,94,8,48,1333,9855,2019-01,아시아,85.0,0.9
9,미얀마,1304,10,31,67,3877,5289,2019-01,아시아,24.7,0.1


In [133]:
# 국가별로 우리나라에 관광을 위해서 들어오는 비율이 가장 높은 5개 국가만 조회...
kto_201901_contry_newindex.sort_values('전체비율(%)', ascending = False).head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객 비율(%),전체비율(%)
17,중국,320113,2993,138,8793,60777,392814,2019-01,아시아,81.5,36.2
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3,22.5
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2,9.8
25,미국,42989,418,2578,229,16523,62737,2019-01,아메리카,68.5,4.9
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5,3.9


In [150]:
title = list()
title

[]

In [151]:
# 201001 ~ 201912 까지의
# 파일명 자동생성하기
# kto_yyyymm.xlsx형태로 출력해 보세요
# 년도 4자리, 월 2자리 입니다.
# 또는 for n in range(2010,2020): 써도 좋음

for n in range(0,10):
    for m in range(1,13):
        title.append('kto_201' + str(n) + str(m).zfill(2))
        
title

['kto_201001',
 'kto_201002',
 'kto_201003',
 'kto_201004',
 'kto_201005',
 'kto_201006',
 'kto_201007',
 'kto_201008',
 'kto_201009',
 'kto_201010',
 'kto_201011',
 'kto_201012',
 'kto_201101',
 'kto_201102',
 'kto_201103',
 'kto_201104',
 'kto_201105',
 'kto_201106',
 'kto_201107',
 'kto_201108',
 'kto_201109',
 'kto_201110',
 'kto_201111',
 'kto_201112',
 'kto_201201',
 'kto_201202',
 'kto_201203',
 'kto_201204',
 'kto_201205',
 'kto_201206',
 'kto_201207',
 'kto_201208',
 'kto_201209',
 'kto_201210',
 'kto_201211',
 'kto_201212',
 'kto_201301',
 'kto_201302',
 'kto_201303',
 'kto_201304',
 'kto_201305',
 'kto_201306',
 'kto_201307',
 'kto_201308',
 'kto_201309',
 'kto_201310',
 'kto_201311',
 'kto_201312',
 'kto_201401',
 'kto_201402',
 'kto_201403',
 'kto_201404',
 'kto_201405',
 'kto_201406',
 'kto_201407',
 'kto_201408',
 'kto_201409',
 'kto_201410',
 'kto_201411',
 'kto_201412',
 'kto_201501',
 'kto_201502',
 'kto_201503',
 'kto_201504',
 'kto_201505',
 'kto_201506',
 'kto_2015

In [199]:
for yy in range(2010,2020):
    for mm in range(1,13):
        file_path = '.files/kto{}{}.xlsx'.format(yy,str(mm).zfill(2))
        df = pd.read_excel(path, header = 1,
              skipfooter = 4,  # 아래에서 행 제거
              usecols = 'A:G') # 읽어들일 컬럼 선택
        print(len(df))

67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67
67


In [2]:
for yy in range(2010, 2020) :
    for mm in range(1, 13) :
        file_path = "./files/kto_{}{}.xlsx".format(yy, str(mm).zfill(2))
        df = pd.read_excel(file_path, header=1, skipfooter=4, usecols="A:G")
        
        df["기준년월"] = "{}-{}".format(yy, str(mm).zfill(2))
        
        continents_list = ["아시아주", "미주", "구주", "대양주",
                    "아프리카주", "기타대륙", "교포소계"]
        
        condition = df["국적"].isin(continents_list)
        df_country = df[condition == False]
        df_country_newindex = df_country.reset_index(drop=True)
        
        # 아시아(25), 아메리카(5), 유럽(23), 오세아니아(3), 
        # 아프리카(2), 기타대륙, 교포
        continents = ["아시아"] * 25 + ["아메리카"] * 5 + ["유럽"] * 23  \
                 + ["오세아니아"] * 3 + ["아프리카"] * 2 + ["기타대륙"] + ["교포"]
        
        df_country_newindex["대륙"] = continents
        df_country_newindex["관광객비율(%)"] = \
                                    round(df_country_newindex["관광"] / \
                                          df_country_newindex["계"] * 100, 1)
        df_country_newindex["전체비율(%)"] = \
                round(df_country_newindex["관광"] / \
                      sum(df_country_newindex["관광"]) * 100, 1)
        
        print(df_country_newindex.head(1))

   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  202825  1750  89    549  3971  209184  2010-01  아시아      97.0     50.6
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  224475  1455  63   1172  3197  230362  2010-02  아시아      97.4     50.3
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  299820  1483  124    519  4180  306126  2010-03  아시아      97.9     55.0
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  233697  1282  74    154  3868  239075  2010-04  아시아      97.8     43.0
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  233311  1480  150    216  6538  241695  2010-05  아시아      96.5     44.4
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  226786  1236  81    358  3257  231718  2010-06  아시아      97.9     43.6
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  

   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  186840  2476  216    629  6145  196306  2014-09  아시아      95.2     19.9
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  182575  2074  481    294  4911  190335  2014-10  아시아      95.9     17.3
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  172285  2155  189    383  4521  179533  2014-11  아시아      96.0     19.1
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  164921  1658  128    392  3388  170487  2014-12  아시아      96.7     18.6
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  131909  2606  110    695  4312  139632  2015-01  아시아      94.5     18.2
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  134366  2464  96   1674  3987  142587  2015-02  아시아      94.2     16.5
   국적      관광    상용   공용  유학/연수    기타       계     기준년월

   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  283101  1584  126    333  4948  290092  2019-04  아시아      97.6     20.5
   국적      관광    상용  공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  279174  2019  91    715  4274  286273  2019-05  아시아      97.5     22.6
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  275286  1901  136    771  4382  282476  2019-06  아시아      97.5     22.2
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  267816  1855  149    694  4316  274830  2019-07  아시아      97.4     22.0
   국적      관광    상용   공용  유학/연수    기타       계     기준년월   대륙  관광객비율(%)  전체비율(%)
0  일본  318985  2195  165   2957  5350  329652  2019-08  아시아      96.8     25.5


FileNotFoundError: [Errno 2] No such file or directory: './files/kto_201909.xlsx'

In [312]:
# del(df_result)
# df_result = list()
df_all = pd.DataFrame()
for yy in range(2010, 2020) :
    for mm in range(1, 13) :
        try:
            file_path = "./files/kto_{}{}.xlsx".format(yy, str(mm).zfill(2))
            df = pd.read_excel(file_path, header=1, skipfooter=4, usecols="A:G")

            #df["기준년월"] = "2019-01"
            df["기준년월"] = "{}-{}".format(yy, str(mm).zfill(2))

            continents_list = ["아시아주", "미주", "구주", "대양주",
                        "아프리카주", "기타대륙", "교포소계"]

            condition = df["국적"].isin(continents_list)
            df_country = df[condition == False]
            df_country_newindex = df_country.reset_index(drop=True)

            # 아시아(25), 아메리카(5), 유럽(23), 오세아니아(3), 
            # 아프리카(2), 기타대륙, 교포
            continents = ["아시아"] * 25 + ["아메리카"] * 5 + ["유럽"] * 23  \
                     + ["오세아니아"] * 3 + ["아프리카"] * 2 + ["기타대륙"] + ["교포"]

            df_country_newindex["대륙"] = continents
            df_country_newindex["관광객비율(%)"] = \
                                        round(df_country_newindex["관광"] / \
                                              df_country_newindex["계"] * 100, 1)
            df_country_newindex["전체비율(%)"] = \
                    round(df_country_newindex["관광"] / \
                          sum(df_country_newindex["관광"]) * 100, 1)
            
            df_all = df_all.append(df_country_newindex, ignore_index=True)
        
        except:
            pass
len(df_all)

6960

In [328]:
# 함수로 만들기
# 함수이름 ㅣ create_kto_data
def create_kto_data():
    df_all = pd.DataFrame()
    for yy in range(2010, 2020) :
        for mm in range(1, 13) :
            try:
                file_path = "./files/kto_{}{}.xlsx".format(yy, str(mm).zfill(2))
                df = pd.read_excel(file_path, header=1, skipfooter=4, usecols="A:G")

                #df["기준년월"] = "2019-01"
                df["기준년월"] = "{}-{}".format(yy, str(mm).zfill(2))

                continents_list = ["아시아주", "미주", "구주", "대양주",
                            "아프리카주", "기타대륙", "교포소계"]

                condition = df["국적"].isin(continents_list)
                df_country = df[condition == False]
                df_country_newindex = df_country.reset_index(drop=True)

                # 아시아(25), 아메리카(5), 유럽(23), 오세아니아(3), 
                # 아프리카(2), 기타대륙, 교포
                continents = ["아시아"] * 25 + ["아메리카"] * 5 + ["유럽"] * 23  \
                         + ["오세아니아"] * 3 + ["아프리카"] * 2 + ["기타대륙"] + ["교포"]

                df_country_newindex["대륙"] = continents
                df_country_newindex["관광객비율(%)"] = \
                                            round(df_country_newindex["관광"] / \
                                                  df_country_newindex["계"] * 100, 1)
                df_country_newindex["전체비율(%)"] = \
                        round(df_country_newindex["관광"] / \
                              sum(df_country_newindex["관광"]) * 100, 1)

                df_all = df_all.append(df_country_newindex, ignore_index=True)

            except:
                pass
        print(len(df_all))

In [365]:
create_kto_data()

720
1440
2160
2880
3600
4320
5040
5760
6480
6960


In [3]:
# 반복문 안에서 # 합치지는 못하고 떨어진 상태로 진행
# 16개의 파일 불러오기
# 파일을 하나씩 불러온 후 df1라는 데이터 프레임 변수에 저장 후
# len(df1)로 가각의 행의 갯수를 출력하게 해주세요. 
# 이미 df1=list() 로 만들어 주고 이미 들어가 있으니 재실행 ㄴㄴ
# 재실행 했으면 del(df1)
# del(df1)
df1_all = pd.DataFrame()
continents_list = ["아시아주", "미주", "구주", "대양주",
                            "아프리카주", "기타대륙", "교포소계"]
for n in title[:-4]:
    path = './files/' + n + '.xlsx'
    df1 = pd.read_excel(path, header = 1,
            skipfooter = 4,  # 아래에서 행 제거
            usecols = 'A:G') # 읽어들일 컬럼 선택
    # 미리 append를 써버리면 데이터가 7000개쯤 쌓여서 데이터 추가할때 힘들어진다
    # 아래의 continents 같은 경우를 말함
    # 미리 append를 한다면 조건문으로 달아줘야할듯
    df1['기준년월'] = n[4:8] + '-' + n[8:]
    
    condition1 = df1["국적"].isin(continents_list)
    df1_country = df1[condition1 == False]
    df1_country_newindex = df1_country.reset_index(drop=True)
    
    continents1 = ["아시아"] * 25 + ["아메리카"] * 5 + ["유럽"] * 23  \
                + ["오세아니아"] * 3 + ["아프리카"] * 2 + ["기타대륙"] + ["교포"]
    
    df1_country_newindex["대륙"] = continents1
    
    df1_country_newindex["관광객비율(%)"] = \
        round(df1_country_newindex["관광"] / \
        df1_country_newindex["계"] * 100, 1)
    
    df1_country_newindex["전체비율(%)"] = \
        round(df1_country_newindex["관광"] / \
        sum(df1_country_newindex["관광"]) * 100, 1)

    df1_all = df1_all.append(df1_country_newindex, ignore_index=True)
    
len(df1_all)
    


NameError: name 'title' is not defined

In [375]:
df1_all.describe()

,관광,상용,공용,유학/연수,기타,계,관광객비율(%),전체비율(%)
count,6960.000000,6960.000000,6960.000000,6960.000000,6960.000000,6960.000000,6960.000000,6960.000000
mean,14151.834914,310.131178,61.251149,303.418822,3383.024713,18209.660776,62.111940,1.665259
std,52351.026147,1043.723255,250.858698,2116.778318,10269.311046,62120.322558,24.763823,5.915490
min,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000
25%,500.000000,17.000000,1.000000,7.000000,219.000000,953.000000,44.300000,0.100000
50%,1218.500000,50.000000,5.000000,26.000000,611.000000,2235.500000,68.600000,0.100000
75%,5805.000000,225.250000,25.000000,85.000000,1854.250000,9773.750000,80.700000,0.700000
max,823016.000000,25012.000000,3763.000000,44291.000000,137898.000000,917519.000000,100.000000,58.600000


In [378]:
df1_all['대륙'].unique()

array(['아시아', '아메리카', '유럽', '오세아니아', '아프리카', '기타대륙', '교포'], dtype=object)

In [381]:
# 최종 전처리 데이터 저장하기
# 파일이름 : kto_total.xslx
df_all.to_excel('./files/kto_total.xlsx', index = False)

In [264]:
df1

[         국적      관광     상용   공용  유학/연수     기타       계     기준년월
 0      아시아주  329131  18238  421   4339  80916  433045  2010-01
 1        일본  202825   1750   89    549   3971  209184  2010-01
 2        대만   35788     41   17     37    516   36399  2010-01
 3        홍콩   13874     55    0     21    595   14545  2010-01
 4       마카오     554      0    0      0      0     554  2010-01
 ..      ...     ...    ...  ...    ...    ...     ...      ...
 62  아프리카 기타     349    594   25     86    296    1350  2010-01
 63     기타대륙      10      3    0      0      0      13  2010-01
 64     국적미상      10      3    0      0      0      13  2010-01
 65     교포소계       0      0    0      0  22537   22537  2010-01
 66       교포       0      0    0      0  22537   22537  2010-01
 
 [67 rows x 8 columns],
          국적      관광     상용   공용  유학/연수     기타       계     기준년월
 0      아시아주  383125  13711  364  33910  84396  515506  2010-02
 1        일본  224475   1455   63   1172   3197  230362  2010-02
 2        대만  